In [109]:
%pip install transformers datasets evaluate accelerate peft tqdm ipywidgets torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
hugging_face_token = ""

In [115]:
%pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from datasets import load_dataset, Dataset
import pandas as pd
from torch.utils.data import DataLoader
import evaluate


In [3]:
neologism_data = pd.read_csv('all_sampled_twitter.csv')
neologism_data['label'] = neologism_data['label'].astype("int64")

dataset = Dataset.from_pandas(neologism_data)

train_test_split = dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]
eval_dataset = train_test_split["test"]

In [4]:
neologism_data

,text,label
0,"""@mattduss @chrislhayes if ISIS does gain a fo...",0
1,"""Sad news! 150 people shot, drowned in Yobe ri...",0
2,@StephenNolan they all served and died under t...,0
3,Rousey vs. Holm may be the biggest waste of a ...,0
4,@user Amy Schumer is the stereotypical 1st wor...,0
...,...,...
8739,The city experienced a complete blackout after...,0
8740,"After the power outage, the city experienced a...",0
8741,The unexpected blackout plunged the city into ...,0
8742,The company's sudden mobilization of resources...,0


In [5]:
base_model = 'roberta-base'

device = "cuda"

num_labels = 3
class_names = ['negative', 'neutral', 'positive']

id2label = {i: label for i, label in enumerate(class_names)}

In [6]:
tokenizer_full_finetune = RobertaTokenizer.from_pretrained(base_model)
model_to_train = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels).to(device)

def tokenize_function(examples):
    return tokenizer_full_finetune(examples["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True).with_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset = test_dataset.map(tokenize_function, batched=True).with_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset = eval_dataset.map(tokenize_function, batched=True).with_format("torch", columns=["input_ids", "attention_mask", "label"])


data_collator = DataCollatorWithPadding(tokenizer=tokenizer_full_finetune, return_tensors="pt")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6120 [00:00<?, ? examples/s]

Map:   0%|          | 0/2624 [00:00<?, ? examples/s]

Map:   0%|          | 0/2624 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    eval_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    output_dir="AkhilaGP/roberta-base-neologism-full-finetune",
    push_to_hub=True
)

def get_trainer(model):
      return  Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

In [8]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def evaluate_model(inference_model, dataset):

    predictions_full = []
    references_full = []
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    inference_model.to("cuda")
    inference_model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to("cuda")
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        predictions_full.append(predictions)
        references_full.append(references)
        accuracy_metric.add_batch(predictions=predictions,references=references,)
        f1_metric.add_batch(predictions=predictions,references=references,)
        precision_metric.add_batch(predictions=predictions,references=references,)
        recall_metric.add_batch(predictions=predictions,references=references,)

    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average="weighted")
    precision = precision_metric.compute(average="weighted")
    recall = recall_metric.compute(average="weighted")

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score (macro): {f1}")
    print(f"Precision (macro): {precision}")
    print(f"Recall (macro): {recall}")
    return predictions_full, references_full

In [11]:
full_finetuning_trainer = get_trainer(model_to_train)
full_finetuning_trainer.train()
full_finetuning_trainer.push_to_hub()
# tokenizer_full_finetune.push_to_hub("AkhilaGP/roberta-base-neologism-full-finetune")

Step,Training Loss,Validation Loss


In [10]:
from tqdm import tqdm
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels).to(device), test_dataset)
pred, ref = evaluate_model(AutoModelForSequenceClassification.from_pretrained("AkhilaGP/roberta-base-neologism-full-finetune", id2label=id2label, num_labels=num_labels).to(device), test_dataset)

100%|██████████| 328/328 [00:25<00:00, 12.85it/s]


Accuracy: {'accuracy': 0.7660060975609756}
F1 Score (macro): {'f1': 0.7637822701624826}
Precision (macro): {'precision': 0.7622441475221033}
Recall (macro): {'recall': 0.7660060975609756}


([tensor([1, 2, 0, 0, 0, 2, 0, 0], device='cuda:0'),
  tensor([0, 1, 2, 0, 2, 1, 0, 0], device='cuda:0'),
  tensor([1, 2, 0, 2, 0, 2, 1, 2], device='cuda:0'),
  tensor([2, 0, 2, 2, 0, 1, 0, 2], device='cuda:0'),
  tensor([2, 1, 0, 2, 1, 0, 1, 0], device='cuda:0'),
  tensor([2, 0, 1, 0, 0, 1, 2, 1], device='cuda:0'),
  tensor([1, 0, 0, 0, 0, 0, 2, 2], device='cuda:0'),
  tensor([0, 2, 0, 0, 2, 0, 2, 0], device='cuda:0'),
  tensor([0, 1, 1, 2, 0, 1, 2, 2], device='cuda:0'),
  tensor([2, 2, 1, 0, 2, 2, 0, 0], device='cuda:0'),
  tensor([0, 1, 0, 0, 0, 2, 2, 0], device='cuda:0'),
  tensor([0, 2, 1, 2, 1, 2, 2, 0], device='cuda:0'),
  tensor([0, 1, 2, 2, 2, 2, 2, 0], device='cuda:0'),
  tensor([1, 1, 0, 2, 0, 1, 0, 2], device='cuda:0'),
  tensor([2, 2, 2, 1, 0, 2, 2, 0], device='cuda:0'),
  tensor([2, 1, 1, 2, 2, 1, 2, 1], device='cuda:0'),
  tensor([0, 0, 0, 0, 2, 2, 2, 0], device='cuda:0'),
  tensor([2, 2, 0, 0, 0, 0, 2, 2], device='cuda:0'),
  tensor([2, 0, 1, 0, 2, 0, 2, 2], device='cud

In [11]:
def classify(text, model, tokenizer):
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
  output = model(**inputs)
  prediction = output.logits.argmax(dim=-1).item()
  print(f'Sentiment: {id2label[prediction]} for {text}')

In [16]:
def classify_og(text, model, tokenizer):
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to(device)
  output = model(**inputs)
  prediction = output.logits.argmax(dim=-1).item()
  print(f'Sentiment: {id2label[prediction]} for {text}')

In [15]:
samples = [    "Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia",
        "That fit is straight fire, no cap, you're looking mad schmick",
        "The candidate's speech was pure clickbait, all sizzle and no steak",
        "The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.",
        "Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.",
        "That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity",
        "Taylor's new album slaps hard"]
        

non_finetuned = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels).to(device)
full_finetuned = AutoModelForSequenceClassification.from_pretrained("AkhilaGP/roberta-base-neologism-full-finetune", id2label=id2label, num_labels=num_labels).to(device)

for i in samples:
    classify(i, non_finetuned , tokenizer_full_finetune )

print("Full")
for i in samples:
    classify(i, full_finetuned, tokenizer_full_finetune)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Sentiment: negative for Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia
Sentiment: negative for That fit is straight fire, no cap, you're looking mad schmick
Sentiment: negative for The candidate's speech was pure clickbait, all sizzle and no steak
Sentiment: negative for The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.
Sentiment: negative for Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.
Sentiment: negative for That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity
Sentiment: negative for Taylor's new album slaps hard
Full
Sentiment: positive for Listening to the retro playlis

LORA on roberta-base

In [16]:
training_args_LORA = TrainingArguments(
    eval_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available(),
    output_dir="AkhilaGP/roberta-base-neologism-LORA",
    push_to_hub=True

)

def get_trainer_LORA(model):
      return  Trainer(
          model=model,
          args=training_args_LORA,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

In [20]:
peft_config = LoraConfig(task_type="SEQ_CLS", 
                         inference_mode=False, 
                         r=8, 
                         lora_alpha=16, 
                         lora_dropout=0.1)

peft_model = get_peft_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label).to(device), 
                            peft_config)

peft_model.print_trainable_parameters()

peft_lora_finetuning_trainer = get_trainer_LORA(peft_model)

peft_lora_finetuning_trainer.train()
# peft_lora_finetuning_trainer.push_to_hub()
# tokenizer_full_finetune.push_to_hub("roberta-base-neologism-LORA")

README.md:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AkhilaGP/roberta-base-neologism-LORA/commit/4da548257216f4b1c7608ec4cefff5db7b487796', commit_message='Upload tokenizer', commit_description='', oid='4da548257216f4b1c7608ec4cefff5db7b487796', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AkhilaGP/roberta-base-neologism-LORA', endpoint='https://huggingface.co', repo_type='model', repo_id='AkhilaGP/roberta-base-neologism-LORA'), pr_revision=None, pr_num=None)

In [22]:
roberta_lora = AutoPeftModelForSequenceClassification.from_pretrained("AkhilaGP/roberta-base-neologism-LORA", id2label=id2label)
tokenizer_lora = AutoTokenizer.from_pretrained("AkhilaGP/roberta-base-neologism-LORA")
pred, ref = evaluate_model(roberta_lora, test_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 328/328 [00:26<00:00, 12.28it/s]


Accuracy: {'accuracy': 0.7397103658536586}
F1 Score (macro): {'f1': 0.7250822680299361}
Precision (macro): {'precision': 0.7252004262404115}
Recall (macro): {'recall': 0.7397103658536586}


In [23]:
for i in samples:
    classify(i, roberta_lora , tokenizer_lora)

Sentiment: positive for Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia
Sentiment: negative for That fit is straight fire, no cap, you're looking mad schmick
Sentiment: negative for The candidate's speech was pure clickbait, all sizzle and no steak
Sentiment: positive for The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.
Sentiment: positive for Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.
Sentiment: negative for That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity
Sentiment: positive for Taylor's new album slaps hard
